# FDTD Yee Algoritmus

Pre riešeni sústavy Maxwellových diferenciálnych rovníc v časovej oblasti predstavil v roku 1966  matematik [Kane Yee](https://en.wikipedia.org/wiki/Finite-difference_time-domain_method) algoritmus, ktorý nesie jeho meno. Podstatou algoritmu je výpočet časových a priestorových derivácii aproximácie derivácie pomocou konečných diferencií.

## Aproximácia priestorových a časových derivácií

Pre popis elektromagnetického poľa v priestore a čase zavedieme označenie, ktoré popisuje diskrétnu časovo-priestorovú lokalizáciu komponentov poľa $\mathbf{E}$ a $\mathbf{H}$

\begin{align*}
\mathbf{E}(\mathbf{r},t) &\approx \mathbf{E}(m \Delta x, n \Delta y, k \Delta z, q \Delta t) = \mathbf{E}^q[m,n,k] \\
\mathbf{H}(\mathbf{r},t) &\approx \mathbf{H}(m \Delta x, n \Delta y, k \Delta z, q \Delta t) = \mathbf{H}^q[m,n,k] \\
\end{align*}

kde $m,n,k$ sú indexy priestorových diferencií v smere osí, $q$ je index časovej diferencie. Deriváciu komponentu poľa v nejakom bode priestoru a čase, napríklad $\mathbf{H}$ v bode $(m \Delta x, n \Delta y, k \Delta z)$ v smere osi $z$ a v čase $q$, potom môžeme formálne aproximovať ako

\begin{equation}
\frac{\partial \mathbf{H}^q[m,n,k]}{\partial z} \approx \frac{\mathbf{H}^q \left[ m,n,k + \frac{1}{2} \right] - \mathbf{H}^q \left[m, n, k- \frac{1}{2} \right ]}{\Delta z} + O[(\Delta z)^2]
\\
\end{equation}

Z aproximácie (*two-point centered difference method*) vyplýva, že zložky poľa by mali byť definované v celočíselných bodoch mriežky (m,n), ako aj v medzi bodmi mriežky v polovičných vzdialenostiach (k+1/2, k-1/2). Derivácie zložky komponenty poľa, napríklad $H_z$ v smere osi $x$ môžeme aproximovať ako

\begin{equation}
\frac{\partial H_z^q [m,n,k]}{\partial z} \approx \frac{ H_z^q \left[ m,n,k + \frac{1}{2} \right] - H_z^q \left[m, n, k- \frac{1}{2} \right ]}{\Delta z} + O[(\Delta z)^2]
\\
\end{equation}

**Poznámka** - Komponenty $\mathbf{E}$ a $\mathbf{H}$ sú vektorové polia, takže každá zložka komponentu poľa je  skalárna funkcia, ktorá môže závisieť od všetkých súradníc. 

Rovnakým postupom môžeme aproximovať aj časové derivácie v Ampérovom a Faradayovom zákone. Pre jednoduchosť nebudeme v nasledujúcom zápise uvádzať priestorové súradnice.

\begin{align*}
\frac{\partial \mathbf{E}^{q+ 1/2}}{\partial t} &\approx \frac{\mathbf{E}^{q+1} - \mathbf{E}^q }{\Delta t} 
= \frac{1}{\epsilon}  \left[ \nabla \times \mathbf{H} \right]^{q+1/2}\\
\\
\frac{\partial \mathbf{H}^{q+ 1}}{\partial t} &\approx \frac{\mathbf{H}^{q+3/2} - \mathbf{H}^{q+1/2} }{\Delta t} 
= \frac{1}{\mu}  \left[ \nabla \times \mathbf{E} \right]^{q+1}
\end{align*}

Je zrejmé, že obe rovnice sú vzájomne previazané a princíp posunutia komponentov $\mathbf{E}$ a $\mathbf{H}$ o polovicu časového intervalu je zrejmý z nasledujúcej úpravy a obrázku

\begin{align*}
\mathbf{E}^{q+1} &= \mathbf{E}^q  + \frac{\Delta t}{\epsilon}  \left[ \nabla \times \mathbf{H} \right]^{q+1/2}\\
\\
\mathbf{H}^{q+3/2} &= \mathbf{H}^{q+1/2} + \frac{\Delta t}{\mu}  \left[ \nabla \times \mathbf{E} \right]^{q+1}
\end{align*}


**TODO OBRAZOK** Yee bunka

## Yee Algoritmus v 1D 

Predpokladajme jednorozmenrný priestor v ktorom sa môže šíríť elektromagnetická vlna v smere osi $x$. Elektromagnetická vlna pozostáva len zo zložiek poľa v smere osí $y$ a $z$, tieto sú funkciou súradnice $x$.

Podľa Faradayovho zákona potom môžeme písať

\begin{equation}
-\mu \frac{\partial \mathbf{H}}{\partial t} = 
-\mu \left[ \frac{\partial H_y}{\partial t}\mathbf{j} + \frac{\partial H_z}{\partial t}\mathbf{k} \right]
= \nabla \times \mathbf{E} = 
\begin{vmatrix} \mathbf{i} & \mathbf{j} & \mathbf{k} \\
{\dfrac{\partial}{\partial x}} & 0 & 0 \\
0 & E_y & E_z \end{vmatrix} =
-\frac{\partial E_z}{\partial x} \mathbf{j}  +  \frac{\partial E_y}{\partial x} \mathbf{k}
\end{equation}

a z Ampérovho zákona vyplýva

\begin{equation}
\epsilon \frac{\partial \mathbf{E}}{\partial t} =
\epsilon \left[ \frac{\partial E_y}{\partial t} \mathbf{j} + \frac{\partial E_z}{\partial t} \mathbf{k} \right]=
\nabla \times \mathbf{H} = 
\begin{vmatrix} \mathbf{i} & \mathbf{j} & \mathbf{k} \\
{\dfrac{\partial}{\partial x}} & 0 & 0 \\
0 & H_y & H_z \end{vmatrix} =
-\frac{\partial H_z}{\partial x}\mathbf{j} + \frac{\partial H_y}{\partial x}\mathbf{k}
\end{equation}


Rozpísaní pre zložky polí dostaneme dve sústavu skalárnych rovníc, jednu pre vzájomne kolmé zložky $H_y$ a $E_z$, označované aj ako TM vlna

\begin{align}
-\mu \frac{\partial H_y}{\partial t}  &= -\frac{\partial E_z}{\partial x} \\
\epsilon \frac{\partial E_z}{\partial t}  &= \frac{\partial H_y}{\partial x}
\end{align}

**TODO** Obrazok TM vlny

a druhú pre zložky $E_y$ a $H_z$, označované aj ako TE vlna

\begin{align}
-\mu \frac{\partial H_z}{\partial t}  &= \frac{\partial E_y}{\partial x} \\
\epsilon \frac{\partial E_y}{\partial t}  &= -\frac{\partial H_z}{\partial x}
\end{align}

**TODO** Obrazok TE vlny

Z rovníc je zrejmé, že časová derivácia jednnej komponenty poľa závisí od priestorovej derivácia druhej komponenty. 

**Poznámka** Pri šírení sa elektromagnetickej vlny vo voľnom priestore sú TE a TM vlny nezávislé, zložky elektrického ako $E_y, E_z$ resp. magnetického poľa $H_y, H_z$ spolu nesúvisia. K zmene ich veľkosti môže dôjsť napríklad pri prechode prostredím, ktoré vykazuje polarizačné vlastnosti a pod.  